In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [2]:
df_ch = pd.read_csv('../Archivos/LastFm/Carlos_scrob.csv')
df_maria = pd.read_csv('../Archivos/LastFm/Maria_scrob.csv')


In [3]:
df_ch=df_ch.drop(['uts','artist_mbid','album_mbid','track_mbid'],axis = 1)
df_ch.columns=['Date','Artist','Album','Song']
df_maria=df_maria.drop(['uts','artist_mbid','album_mbid','track_mbid'],axis = 1)
df_maria.columns=['Date','Artist','Album','Song']

In [4]:
percent_missing_ch=df_ch.isnull().sum()*100/len(df_ch)
percent_missing_maria=df_maria.isnull().sum()*100/len(df_maria)
df_missing_value=pd.DataFrame(
    {
    'Ch':percent_missing_ch,
    'Maria':percent_missing_maria,
    })
df_missing_value

,Ch,Maria
Date,0.000000,0.000000
Artist,0.000000,0.000000
Album,7.886792,1.818633
Song,0.000000,0.000000


In [5]:
df_ch['Date']=pd.to_datetime(df_ch['Date'])
df_ch['Year']=pd.DatetimeIndex(df_ch['Date']).year
df_maria['Date']=pd.to_datetime(df_maria['Date'])
df_maria['Year']=pd.DatetimeIndex(df_maria['Date']).year


In [6]:
df_ch['User']='Ch'
df_maria['User']= 'Maria'


In [7]:
df_all= pd.concat([df_ch, df_maria], axis=0, ignore_index=True)
df_all.tail(3)

,Date,Artist,Album,Song,Year,User
316289,2009-08-23 13:51:00,The Killers,Hot Fuss,Somebody Told Me,2009,Maria
316290,2009-08-23 13:48:00,The Killers,Day & Age,This Is Your Life,2009,Maria
316291,2009-08-23 13:44:00,The Killers,Day & Age,Human,2009,Maria


In [8]:
ChbyYear=df_ch.groupby(df_ch.Year).Song.count()
df_ChbyYear=ChbyYear.to_frame()
df_ChbyYear['User']='Ch'
df_ChbyYear['Year']= ChbyYear.index
df_ChbyYear.reset_index(drop=True, inplace=True)


In [10]:
MariabyYear=df_maria.groupby(df_maria.Year).Song.count()
df_MariabyYear=MariabyYear.to_frame()
df_MariabyYear['User']='Maria'
df_MariabyYear['Year']= MariabyYear.index
df_MariabyYear.reset_index(drop=True, inplace=True)

In [11]:
df_concatenated = pd.concat([df_ChbyYear, df_MariabyYear])

In [12]:
px.line(df_concatenated ,
        x = 'Year',
        y = 'Song',
        color = 'User')

In [13]:
x=2008
dfs=[]
while x<2022:
    year_tmp=df_ch[df_ch['Year']==x]
    year_tmp=year_tmp.groupby('Artist').Artist.count()
    df_temp1=year_tmp.to_frame()
    df_temp1.columns=['Count']
    df_temp1['Artist']= year_tmp.index
    df_temp1.reset_index(drop=True, inplace=True)
    df_temp2=df_temp1.sort_values('Count', ascending=False).head(5)
    df_temp2['Year']=x
    df_temp2.reset_index(drop=True, inplace=True)
    dfs.append(df_temp2)
    x=x+1
df_ChbyArtistYear=pd.concat(dfs, ignore_index=True)
Ch_fig=px.histogram(df_ChbyArtistYear,
        x = 'Count',
        y = 'Artist',
        color='Artist',
        animation_frame='Year')
Ch_fig2 = go.Figure()
for tr in Ch_fig.frames[-1].data:
    Ch_fig2.add_trace(tr)
Ch_fig2.layout = Ch_fig.layout
Ch_fig2.frames = Ch_fig.frames
Ch_fig2.layout['sliders'][0]['active'] = len(Ch_fig.frames) - 1
Ch_fig2

In [14]:
x=2008
dfs=[]
while x<2022:
    year_tmp=df_maria[df_maria['Year']==x]
    year_tmp=year_tmp.groupby('Artist').Artist.count()
    df_temp1=year_tmp.to_frame()
    df_temp1.columns=['Count']
    df_temp1['Artist']= year_tmp.index
    df_temp1.reset_index(drop=True, inplace=True)
    df_temp2=df_temp1.sort_values('Count', ascending=False).head(10)
    df_temp2['Year']=x
    df_temp2.reset_index(drop=True, inplace=True)
    dfs.append(df_temp2)
    x=x+1
df_MariabyArtistYear=pd.concat(dfs, ignore_index=True)
maria_fig=px.histogram(df_MariabyArtistYear,
        x = 'Count',
        y = 'Artist',
        color='Artist',
        animation_frame='Year')
maria_fig2 = go.Figure()
for tr in maria_fig.frames[-1].data:
    maria_fig2.add_trace(tr)
maria_fig2.layout = maria_fig.layout
maria_fig2.frames = maria_fig.frames
maria_fig2.layout['sliders'][0]['active'] = len(maria_fig.frames) - 1
maria_fig2